# ETH Price Prediction using Hidden Markov Model (HMM)
In this notebook, we will use a Hidden Markov Model to predict the price of Ethereum (ETH) over the next week. We will use historical price data to train our model.

In [ ]:
# Importing Required Libraries
!pip install hmmlearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from hmmlearn import hmm
import seaborn as sns
import requests
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

sns.set()

In [ ]:
# Function to fetch ETH price data from Coingecko API
def fetch_price_data(days):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)
    url = f'https://api.coingecko.com/api/v3/coins/ethereum/market_chart/range?vs_currency=usd&from={start_date.timestamp()}&to={end_date.timestamp()}'
    response = requests.get(url)
    data = response.json()
    prices = data['prices']
    price_data = pd.DataFrame(prices, columns=['time', 'price'])
    price_data['time'] = pd.to_datetime(price_data['time'], unit='ms')
    return price_data

# Fetching last 365 days of data
price_data = fetch_price_data(365)
price_data.head()

In [ ]:
# Plotting the price data
plt.figure(figsize=(10,5))
plt.plot(price_data['time'], price_data['price'])
plt.title('ETH Price Data (Last 365 Days)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.show()

In [ ]:
# Preprocessing the data for HMM
def preprocess_data(data):
    data = data.copy()
    data['log_return'] = np.log(data['price']).diff()
    data = data.dropna()
    return data

processed_data = preprocess_data(price_data)
processed_data.head()

In [ ]:
# Training the HMM
model = hmm.GaussianHMM(n_components=4, covariance_type='diag', n_iter=1000)
model.fit(processed_data['log_return'].values.reshape(-1, 1))

# Predicting the hidden states
hidden_states = model.predict(processed_data['log_return'].values.reshape(-1, 1))

In [ ]:
# Plotting the hidden states
plt.figure(figsize=(10,5))
plt.plot(processed_data['time'], hidden_states)
plt.title('Hidden States')
plt.xlabel('Date')
plt.ylabel('State')
plt.show()

In [ ]:
# Function to simulate the next 7 days
def simulate_next_7_days(model, last_state):
    future_days = 7
    predictions = []
    current_state = last_state

    for _ in range(future_days):
        next_state = np.random.choice(4, p=model.transmat_[current_state])
        next_observation = np.random.normal(loc=model.means_[next_state], scale=np.sqrt(model.covars_[next_state]))
        predictions.append(next_observation[0])
        current_state = next_state

    return predictions

# Simulating the next 7 days
predictions = simulate_next_7_days(model, hidden_states[-1])
predictions

In [ ]:
# Converting log returns to price
last_price = price_data['price'].values[-1]
predicted_prices = last_price * np.exp(np.cumsum(predictions))

# Printing the predicted prices
predicted_prices